# Example 2: Feature Space Exploration

This notebook explores how **feature engineering** shapes the selection process. We chain statistical summaries with PCA dimensionality reduction, then use the full suite of feature-space diagnostics to understand the resulting representation.

Key concepts introduced:
- **FeaturePipeline**: chaining multiple feature engineers
- **PCA**: reducing high-dimensional features to interpretable components
- **Multi-objective selection**: balancing Wasserstein fidelity, correlation preservation, and centroid balance
- **KMedoids cluster-size weights**: non-uniform representation

In [1]:
import pandas as pd
import energy_repset as rep
import energy_repset.diagnostics as diag
import plotly.io as pio; pio.renderers.default = 'notebook_connected'

In [2]:
url = "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
df_raw = pd.read_csv(url, index_col=0, parse_dates=True).rename_axis('variable', axis=1)
df_raw = df_raw.drop('prices', axis=1)

slicer = rep.TimeSlicer(unit="month")
context = rep.ProblemContext(df_raw=df_raw, slicer=slicer)
print(f"{len(context.get_unique_slices())} candidate monthly slices")

12 candidate monthly slices


## Feature engineering: Statistics + PCA

`StandardStatsFeatureEngineer` computes ~28 features per month (7 statistics x 4 variables). That's a lot of dimensions for only 12 data points. `PCAFeatureEngineer` then reduces this to the principal components that capture the most variance — a common and effective way to avoid the curse of dimensionality while retaining the essential structure.

In [3]:
feature_pipeline = rep.FeaturePipeline(
    engineers={
        'stats': rep.StandardStatsFeatureEngineer(),
        'pca': rep.PCAFeatureEngineer(),
    }
)
context = feature_pipeline.run(context)
context.df_features

,mean__load,mean__onwind,mean__offwind,mean__solar,std__load,std__onwind,std__offwind,std__solar,q10__load,q10__onwind,...,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,pc_8,pc_9,pc_10,pc_11
2015-01,0.790427,1.623676,1.227180,-1.400283,0.552545,1.875624,1.933594,-1.648768,0.623443,0.404962,...,-0.653615,0.223279,2.176451,0.528092,-1.228347,-0.183354,-0.257430,-0.048796,0.325023,-3.152248e-16
2015-02,1.454759,-0.178153,0.032890,-0.779126,-1.266598,-0.386469,0.387692,-0.710218,1.967376,0.269301,...,-1.649667,2.709401,0.198269,0.196758,0.244496,0.448877,-0.063362,-0.222535,-0.422197,-3.152248e-16
2015-03,0.845602,0.381202,0.172626,-0.018479,-1.705158,0.914114,0.375525,0.222239,1.098170,-0.158519,...,-3.265472,0.251567,-0.904594,0.358105,0.903592,-0.732584,-0.061243,0.703602,0.263503,-3.152248e-16
2015-04,-0.821497,-0.410792,-0.788796,1.006732,-0.171534,-0.065897,-0.749381,1.139187,-0.691634,-0.366475,...,-0.116559,-0.684499,1.364489,1.313323,0.951812,-0.048926,0.797301,-0.408477,-0.096771,-3.152248e-16
2015-05,-1.616257,-0.436483,-0.190973,0.882676,0.209529,-0.537080,-0.411165,0.665516,-1.516858,-0.534456,...,1.534711,0.538564,1.036596,-0.953347,0.418907,-0.319133,-0.886650,0.221965,-0.361881,-3.152248e-16
2015-06,-0.895552,-0.875748,-0.975762,1.083824,-0.187698,-0.689056,-0.947259,0.904855,-0.871983,-0.856384,...,0.590968,-0.152467,-0.316416,0.503298,-1.125940,0.904509,0.405020,0.784066,-0.146887,-3.152248e-16
2015-07,-0.278689,-0.169317,-0.397360,1.176604,-0.130131,0.142227,0.091415,0.993381,-0.131115,-0.524675,...,-1.127886,0.872675,-0.700549,-1.432945,-0.011517,0.811935,0.270986,-0.424934,0.456458,-3.152248e-16
2015-08,-1.004401,-1.163200,-1.165308,0.932397,-0.754763,-1.116381,-1.240157,1.069048,-0.752215,-0.954196,...,-0.008774,-0.607658,-1.439958,1.106555,-0.528968,-0.229647,-0.922617,-0.491525,0.138650,-3.152248e-16
2015-09,-0.324219,-0.398428,-0.400352,0.113524,-0.011197,-0.520080,0.176007,0.080363,-0.535059,0.463223,...,0.338338,-0.044285,-0.626711,-0.947130,-0.912044,-1.367404,0.753570,-0.176294,-0.215180,-3.152248e-16
2015-10,0.183329,-1.215510,-0.785281,-0.640667,0.369950,-1.432897,-1.248423,-0.310932,0.112725,-0.929955,...,1.411115,-1.246839,1.195762,-0.855323,0.645525,0.117859,-0.060932,0.168289,0.314012,-3.152248e-16


### How much variance does each component explain?

In [4]:
fig = diag.PCAVarianceExplained(feature_pipeline.engineers['pca']).plot(show_cumulative=True)
fig.update_layout(title='PCA Variance Explained')
fig.show()

### Feature space: 2D and 3D scatter

Projecting the 12 months into PCA space reveals their relative similarity. Months that are close together have similar statistical profiles; distant months represent different conditions.

In [5]:
fig = diag.FeatureSpaceScatter2D().plot(context.df_features, x='pc_0', y='pc_1')
fig.update_layout(title='Feature Space: PC0 vs PC1')
fig.show()

In [6]:
fig = diag.FeatureSpaceScatter3D().plot(context.df_features, x='pc_0', y='pc_1', z='pc_2')
fig.update_layout(title='Feature Space: 3D PCA Projection')
fig.show()

### Feature correlations and distributions

In [7]:
fig = diag.FeatureCorrelationHeatmap().plot(context.df_features, method='pearson')
fig.update_layout(title='Feature Correlation Matrix')
fig.show()

In [8]:
fig = diag.FeatureSpaceScatterMatrix().plot(
    context.df_features, dimensions=['pc_0', 'pc_1', 'pc_2', 'pc_3']
)
fig.update_layout(title='PCA Scatter Matrix')
fig.show()

In [9]:
fig = diag.FeatureDistributions().plot(context.df_features, nbins=20, cols=4)
fig.update_layout(title='Feature Distributions')
fig.show()

## Define objectives and run the search

We use three objectives to capture different facets of representativeness:

- **Wasserstein fidelity**: do the marginal distributions match?
- **Correlation fidelity**: are cross-variable dependencies preserved?
- **Centroid balance**: is the selection centered in the data cloud (avoiding systematic bias)?

The `ParetoMaxMinStrategy` selects the Pareto-optimal combination that maximizes the *worst-performing* objective — a balanced, conservative choice.

In [10]:
objective_set = rep.ObjectiveSet({
    'wasserstein': (0.5, rep.WassersteinFidelity()),
    'correlation': (0.5, rep.CorrelationFidelity()),
    'centroid_balance': (0.5, rep.CentroidBalance()),
})

k = 3
search_algorithm = rep.ObjectiveDrivenCombinatorialSearchAlgorithm(
    objective_set,
    rep.ParetoMaxMinStrategy(),
    rep.ExhaustiveCombiGen(k=k),
)
representation_model = rep.KMedoidsClustersizeRepresentation()

In [11]:
workflow = rep.Workflow(feature_pipeline, search_algorithm, representation_model)
experiment = rep.RepSetExperiment(context, workflow)
result = experiment.run()

print(f"Selected months: {result.selection}")
print(f"Weights: {result.weights}")
print(f"Scores: {result.scores}")

Iterating over combinations: 100%|██████████| 220/220 [00:01<00:00, 194.75it/s]


Selected months: (Period('2015-01', 'M'), Period('2015-04', 'M'), Period('2015-09', 'M'))
Weights: {Period('2015-01', 'M'): 0.25, Period('2015-04', 'M'): 0.3333333333333333, Period('2015-09', 'M'): 0.4166666666666667}
Scores: {'wasserstein': 0.20025433806827211, 'correlation': 0.0231217738568774, 'centroid_balance': 2.1640052135891987}


## Result diagnostics

### Responsibility weights

Unlike the uniform model in Example 1, the KMedoids representation assigns weights proportional to how many months each representative "covers". The dashed line shows where the bars would be for uniform weighting.

In [12]:
fig = diag.ResponsibilityBars().plot(result.weights, show_uniform_reference=True)
fig.update_layout(title='Responsibility Weights (KMedoids Cluster-Size)')
fig.show()

### Selection in feature space

The selected months (highlighted) should be well-spread across the feature space — covering the range of conditions rather than clustering in one corner.

In [13]:
fig = diag.FeatureSpaceScatter2D().plot(
    context.df_features, x='pc_0', y='pc_1', selection=result.selection
)
fig.update_layout(title='Feature Space with Selected Months')
fig.show()

## Score component diagnostics

These plots assess how well the selected months reproduce specific properties of the full year.

### Distribution fidelity (ECDF)

The empirical CDF of the selection should track the full-year curve closely. Gaps indicate that the selection under- or over-represents certain value ranges.

In [14]:
selected_indices = context.slicer.get_indices_for_slice_combi(context.df_raw.index, result.selection)
df_selection = context.df_raw.loc[selected_indices]

fig = diag.DistributionOverlayECDF().plot(context.df_raw['load'], df_selection['load'])
fig.update_layout(title='Distribution Fidelity: Demand (ECDF)')
fig.show()

### Correlation preservation

The heatmap shows the *difference* between the correlation matrix of the selection and the full year. Values near zero (light) mean the selection preserves that relationship well.

In [15]:
fig = diag.CorrelationDifferenceHeatmap().plot(
    context.df_raw, df_selection, method='pearson', show_lower_only=True
)
fig.update_layout(title='Correlation Difference: Selection - Full Year')
fig.show()

### Diurnal profiles

Comparing the average hourly shape (hour 0-23) of each variable between the full year and the selection. Good matches here mean the selection captures typical within-day patterns.

In [16]:
fig = diag.DiurnalProfileOverlay().plot(
    context.df_raw, df_selection, variables=['load', 'onwind', 'offwind', 'solar']
)
fig.update_layout(title='Diurnal Profiles: Full Year vs Selection')
fig.show()